In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scipy.optimize import minimize


In [ ]:
#Carga de datos
Ly, Lx = 14, 10 
#rho_real = np.load

np.random.seed(0)
dimension = (Ly,Lx)
rho_sim = np.random.exponential(scale=1, size=(dimension, 2))
#rho_sim = LB_AD(D,S)

In [ ]:
def MSE(rho_sim, rho_real):
    error = np.sum((rho_real - rho_sim)**2)
    return error

In [ ]:
result = minimize(MSE,method="BFGS")

In [ ]:
D_opt, S_opt = result.x

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Cargar rho_real desde tus datos reales
rho_real = np.load('ruta_a_tus_datos_rho_real.npy')  # Carga tus datos reales de rho
velocidad_aire = np.load('ruta_a_tus_datos_velocidad_aire.npy')  # Carga tus datos de velocidad del aire

# Función de simulación (rho_sim) que depende de D y S
def simulacion_lbm(D, S, velocidad_aire):
    # Aquí iría tu código de simulación real que produce rho_sim basado en D, S y velocidad del aire
    rho_simulado = 0.7 * D + 0.3 * S  # Este es un placeholder, debes reemplazarlo con tu simulación real
    return rho_simulado

# Generar datos de entrenamiento
num_datos = len(rho_real)
D_inicial = np.random.rand(num_datos, 1) * 0.1  # Valores iniciales de D
S_inicial = np.random.rand(num_datos, 1) * 0.1  # Valores iniciales de S

# Crear una red neuronal para ajustar D y S
model = Sequential([
    Dense(64, input_shape=(3,), activation='relu'),  # 3 entradas: densidad de polución, velocidad del aire, y rho_sim inicial
    Dense(64, activation='relu'),
    Dense(2)  # Dos salidas: D y S
])

# Compilar el modelo
model.compile(optimizer='adam', loss='mse')

# Entrenamiento
for epoch in range(100):
    rho_sim = simulacion_lbm(D_inicial, S_inicial, velocidad_aire)
    entradas = np.hstack((rho_real, velocidad_aire, rho_sim))
    parametros = model.predict(entradas)
    
    # Actualizar D y S basados en las predicciones de la red
    D_inicial, S_inicial = parametros[:, 0], parametros[:, 1]
    
    # Recalcular rho_sim con los nuevos valores de D y S
    rho_sim = simulacion_lbm(D_inicial, S_inicial, velocidad_aire)
    
    # Re-entrenar la red neuronal con los nuevos datos simulados
    model.fit(entradas, np.hstack((D_inicial, S_inicial)), epochs=1, batch_size=32, verbose=1)

# Una vez entrenada la red, podemos hacer predicciones para nuevos datos
densidad_polucion_test = np.array([[0.7]])
velocidad_aire_test = np.array([[0.3]])
rho_sim_test = simulacion_lbm(D_inicial, S_inicial, velocidad_aire_test)
entradas_test = np.hstack((densidad_polucion_test, velocidad_aire_test, rho_sim_test))

parametros_estimados = model.predict(entradas_test)

# Mostrar resultados
print(f"Para una densidad de polución de {densidad_polucion_test[0][0]} y velocidad de aire de {velocidad_aire_test[0][0]}:")
print(f"Constante de difusión estimada (D): {parametros_estimados[0][0]:.5f}")
print(f"Parámetro de fuente estimado (S): {parametros_estimados[0][1]:.5f}")

# Calcular rho_simulado final con los parámetros optimizados
D_estimado, S_estimado = parametros_estimados[0]
rho_simulado_final = simulacion_lbm(D_estimado, S_estimado, velocidad_aire_test)

# Verificar el error final
mse_final = np.mean((rho_real - rho_simulado_final) ** 2)
print(f"MSE final entre rho_real y rho_simulado: {mse_final}")
